In [2]:
import ast
import pandas as pd
import numpy as np

nodes_df = pd.read_csv('nodes.csv' , header = None , names=['node_attribute'], index_col=0)
edges_df = pd.read_csv('edges.csv')


nodes_list = [(index, ast.literal_eval(row.node_attribute))  for index, row in nodes_df.iterrows()]
edges_list = [(row.From, row.To) for index, row in edges_df.iterrows()]

number_of_days = 1448
nodes_list_vec = []
for index,dict_node in nodes_list:
    vec = np.zeros(number_of_days,dtype=np.float32)
    for day, count in dict_node.items():
        vec[day] = count

    nodes_list_vec.append((index, vec))

import networkx as nx
import matplotlib.pyplot as plt


Graph = nx.DiGraph()
for node_id, node_attr_vec in nodes_list_vec:
    Graph.add_node(node_id, x = node_attr_vec)
Graph.add_edges_from(edges_list)

print(f"Number of nodes: {nx.number_of_nodes(Graph)}")
print(f"Number of edges: {nx.number_of_edges(Graph)}")


Number of nodes: 6600
Number of edges: 50897


In [ ]:
import torch
from torch_geometric.utils import from_networkx 

data = from_networkx(Graph)


import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch_geometric.nn import SAGEConv

class GraphSage(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSage, self ).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels )
        self.conv2= SAGEConv(hidden_channels , out_channels)


    def forward(self, x, edge_index):
        x = self.conv1(x , edge_index)
        x =F.relu(x)
        x = self.conv2(x, edge_index)
        return x 


in_channels = data.x.shape[1]       # 1448 in your dataset
hidden_channels = 128
out_channels = 2                     # e.g., for binary classification

model = GraphSage(in_channels, hidden_channels, out_channels)




tensor([5052, 1279, 1466, 3234, 6118, 2902, 4526, 5848, 2442, 1053])
